In [10]:
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from scipy.integrate import quad, simpson
from scipy.interpolate import interp1d

import astropy.constants as c
import astropy.units as u
from dustpy import Simulation, plot
import matplotlib as mpl
import numpy as np
from simframe.frame import Field

mpl.use('macosx')
# mpl.use('module://matplotlib_inline.backend_inline')

In [2]:
m_disk = 0.11 * c.M_sun.cgs.value
m_disk

2.187250857767856e+32

### Simulation gas profile

Use the CO brightness profile as the gas profile for the simulation. First load from file the observed profile, then interpolate it.

In [3]:
fname_profile = Path('./profile_C18O.pickle')

with open(fname_profile, 'rb') as f:
    profile_C18O = pickle.load(f)
    
f, ax = plt.subplots()
ax.plot(profile_C18O['x'] * u.cm.to(u.au), profile_C18O['y'])


def interp_profile_norm(radius: float, profile_dict: dict, total_disk_mass: float) -> float:
    """
    Interpolate a profile to use as gas surface density, normalized to a given total disk mass.
    
    Parameters
    ----------
    radius: float
        Radius where to evaluate the gas surface density.
    profile_dict: dict
        Dictionary containing the profile to use as surface density distribution. The keys must be named 'x' and 'y'.
    total_disk_mass: float
        Total disk mass.
        
    Returns
    -------
    float
        Normalized gas surface density at radius 'radius'.
    """
    
    normalization_constant = total_disk_mass / simpson(2 * np.pi * profile_dict['x'] * profile_dict['y'], profile_dict['x'])
    interp_profile = interp1d(profile_dict['x'], profile_dict['y'], fill_value='extrapolate', kind='quadratic')
    
    return normalization_constant * interp_profile(radius)


Create the radial grid through the Simulation class to compute sigma_mas. This breaks the class methods, so plug the 
sigma_gas into a new class instance.

In [4]:
sim = Simulation()
sim.ini.grid.rmax = profile_C18O['x'].max()
sim.ini.gas.Mdisk = m_disk
sim._makeradialgrid()


In [5]:
SigmaGas = np.array(interp_profile_norm(sim.grid.r, profile_C18O, sim.ini.gas.Mdisk))
sigma_gas = Field(sim, SigmaGas, description="Surface density [g/cm²]")


In [6]:
sim = Simulation()
sim.ini.grid.rmax = profile_C18O['x'].max()
sim.ini.gas.Mdisk = m_disk
sim.gas.Sigma = sigma_gas
sim.initialize()
del(sim.integrator.instructions[1])


In [7]:
model_dir = Path("data_C18O")
sim.writer.datadir = str(model_dir)
if not model_dir.exists():
    sim.run()


### <span style="color:red">Testing area</span>

In [8]:
sim.ini.dust

namespace(aIniMax=0.0001,
          allowDriftingParticles=False,
          erosionMassRatio=10.0,
          d2gRatio=0.01,
          distExp=-3.5,
          excavatedMass=1.0,
          fragmentDistribution=-1.8333333333333333,
          rhoMonomer=1.67,
          vfrag=100.0)

In [9]:
plot.panel('data_mod_profile', im=10, it=20)

RuntimeError: Unknown data type.

In [ ]:
plot.ipanel('data_CO_profile')

In [ ]:
sim_test = Simulation()
sim_test.initialize()
# sim_test.run()
sim_test.ini.gas

In [ ]:
plt.plot(sim_test.grid.r, sim_test.gas.Sigma)
plt.plot(sim.grid.r, sim.gas.Sigma)
plt.xlim(0, (200 * u.au).to_value(u.cm))
plt.ylim(0, 300)

from scipy.integrate import simpson
simpson(sim_test.gas.Sigma * 2 * np.pi * sim_test.grid.r, sim_test.grid.r), simpson(sim.gas.Sigma * 2 * np.pi * 
                                                                                   sim.grid.r, sim.grid.r)

In [ ]:
plt.plot(sim.grid.r * u.cm.to(u.au), sim.gas.Sigma)
# plt.ylim((10, 40))
# plt.ylim((10, 40))
# plt.xlim(20, 120)